## This notebook is for a new approach for regression models to get a better tile-score and a better representation
separating damage features and take only very high or very low damaged slides in terms of individual damages

 A) inflammatory cells such as Neutrophils, Macrophages, and Lymphocytes compiled in  interstitial spaces 

In [3]:
## We have one feature score sheet and also a total score sheet

import pandas as pd
import numpy as np

data_scores_features = pd.read_excel('../../../data/raw_data/ROW_DATA_Sonja_add_MV_ECMO_and_ECMO_LPS.xlsx',sheet_name='Data from all the features ')
data_scores_features

,Sample,PDF/ EXCEL correspodent Data,Intermediate1,Intermediate2,Novice,Expert1,Expert2,Avg
0,Control,10,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,a,1.0,4.0,0.0,3.0,0.0,1.6
2,NaN,b,2.0,3.0,0.0,0.0,0.0,1.0
3,NaN,c,0.0,1.0,1.0,0.0,0.0,0.4
4,NaN,d,0.0,4.0,1.0,4.0,2.0,2.2
...,...,...,...,...,...,...,...,...
400,NaN,d,1.0,1.0,0.0,0.0,0.0,0.4
401,NaN,e,0.0,0.0,0.0,0.0,0.0,0.0
402,NaN,f,0.0,0.0,0.0,0.0,0.0,0.0
403,NaN,g,0.0,0.0,0.0,1.0,0.0,0.2


In [4]:
data_scores_total_score = pd.read_excel('../../../data/raw_data/ROW_DATA_Sonja_add_MV_ECMO_and_ECMO_LPS.xlsx',sheet_name='Total Score')
data_scores_total_score

,PDF/ EXCEL correspodent Data,(sanned slide) Sample Name,Group,Sandra Intermediate,Martin Intermediate,Sinem novice,Oscar Expert,Deniz Expert,min,max,Median,Average,STDEV.P
0,10,9,Control,3,12,4,11,6,3,12,6,7.2,3.655133
1,15,14,Control,3,7,8,11,18,3,18,8,9.4,5.003998
2,16,15,Control,2,9,10,2,5,2,10,5,5.6,3.382307
3,17,16,Control,2,14,6,8,7,2,14,7,7.4,3.878144
4,18,17,Control,3,11,0,5,6,0,11,5,5.0,3.633180
5,19,18,Control,2,9,4,7,15,2,15,7,7.4,4.498889
6,20,19,Control,10,12,9,18,16,9,18,12,13.0,3.464102
7,21,20,Control,3,15,4,9,10,3,15,9,8.2,4.354308
8,23,28,Control,0,10,1,9,7,0,10,7,5.4,4.127953
9,27,26,Control,0,12,10,2,2,0,12,2,5.2,4.833218


In [5]:
A = dict()  ## define an empty dictionary

In [7]:
import pandas as pd
import numpy as np
import shutil
import glob
import os
import subprocess

data_scores_features = pd.read_excel('../../../data/raw_data/ROW_DATA_Sonja_add_MV_ECMO_and_ECMO_LPS.xlsx',sheet_name='Data from all the features ')
data_scores_features
## Here the scores come from a weighted average score of five pathologists with higher importance from experts
weights_score   = np.array([2,2,1,2.5,2.5])
data_scores_features['Weighted_avg'] = 1/(np.sum(weights_score))*(data_scores_features['Intermediate1']*2+\
                                                        data_scores_features['Intermediate2']*2+\
                                                        data_scores_features['Novice']+\
                                                        data_scores_features['Expert1']*2.5+\
                                                        data_scores_features['Expert2']*2.5)
highest_A = (data_scores_features.loc[data_scores_features['PDF/ EXCEL correspodent Data']=='a','Weighted_avg'])
highest_A.index
highest_A.values

####### Here we took only high scores of feature A and also very lowest score to be sure that we have or we have not the features 
a_dominant = [i for i,v in zip(highest_A.index,highest_A.values) if v<2]
a_dominant = np.array(a_dominant).astype(int)-1

for ad in a_dominant:
    print(data_scores_features.iloc[[ad],0:2].values[0])
    ind       = data_scores_features.iloc[[ad],0:2].index[0]
    gr        = data_scores_features.iloc[[ad],0:2].values[0][0]
    nr        = data_scores_features.iloc[[ad],0:2].values[0][1]
    slide_nr  =(data_scores_total_score.loc[data_scores_total_score['PDF/ EXCEL correspodent Data']==nr,'(sanned slide) Sample Name ']).values[0]
    print(slide_nr)
    score_a   = data_scores_features['Weighted_avg'][ind+1]
    score_a   = round(score_a,2)
    if gr in A.keys():
        A[gr]  = np.append(A[gr],slide_nr)
    else:
        A[gr] = slide_nr  
    dst_path = r"../../../data/raw_data/Features/A_prime/"+gr+'/'
    src = r"../../../data/raw_data/Figures_high_magnification_250121/"+str(gr)+'\slide_'+str(slide_nr)+'_*/'
    for filename in glob.glob(os.path.join(src, '*')):
        new_name = filename.split('/')[-1].split('\\')[-1]
        new_n    = new_name.replace('.tif','_score_'+str(score_a)+'.tif').replace('.TIF','_score_'+str(score_a)+'.tif')
        subprocess.call(["cp " ,filename , dst_path+new_n]) #remove () from Slide_9 names
        
print(A)  # I have separated in A ,B, ...
        

['Control' 10]
9
['Control' 15]
14
['Control' 16]
15
['Control' 17]
16
['Control' 18]
17
['Control' 19]
18
['Control' 21]
20
['Control' 23]
28
['Control' 27]
26
['Control' 46]
46
['Control' 47]
47
['MV' 11]
10
['ECMO' 33]
33
['MV' 34]
34
['ECMO' 49]
49
{'MV+LPS': array([35, 42, 44, 45], dtype=int64), 'ECMO+LPS': array([ 1,  5,  6,  7,  8, 25, 37, 48], dtype=int64), 'ECMO': array([39, 33, 49], dtype=int64), 'Control': array([ 9, 14, 15, 16, 17, 18, 20, 28, 26, 46, 47], dtype=int64), 'MV': array([10, 34], dtype=int64)}


In [1]:
### This is for making folds in a random way to have all groups in all folds

def make_folds_from_dict(dict_A, k_fold,new_dict):
    new_dict = {}
    for k in dict_A.keys():
        a = dict_A[k]
        split1 = int(1/k_fold * len(a))
        split2 = int(2/k_fold * len(a))
        np.random.shuffle(a)
        p1 = a[:split1]
        p2 = a[split1:split2]
        p3 = a[split2:]
        new_dict[k] = [p1,p2,p3]
    return new_dict


In [8]:
new_dict = {}
new_dict = make_folds_from_dict(A, 3,new_dict)
print(new_dict)  ## Here we run it once and separate data into folds

{'MV+LPS': [array([35], dtype=int64), array([44], dtype=int64), array([45, 42], dtype=int64)], 'ECMO+LPS': [array([48,  7], dtype=int64), array([25,  6,  5], dtype=int64), array([37,  1,  8], dtype=int64)], 'ECMO': [array([33], dtype=int64), array([39], dtype=int64), array([49], dtype=int64)], 'Control': [array([16, 18, 47], dtype=int64), array([46, 14, 17, 20], dtype=int64), array([28, 26, 15,  9], dtype=int64)], 'MV': [array([], dtype=int64), array([10], dtype=int64), array([34], dtype=int64)]}


In [32]:
import numpy as np
from sklearn.model_selection import train_test_split
import shutil, os
import glob

##Data_12_separated is all the new images in validation dataset

## Here images receive a label according to their scoring and the damage boundaries
## Then images are sorted into train and validation sets (cross-validation is used)

validation_slides = ['1st','2nd','3rd']
Sample_Slide     = new_dict
fold_dict        = {}
for val_dir in validation_slides:
    ind    = validation_slides.index(val_dir)
    fold_dict[val_dir]  = {k:new_dict[k][ind] for k in new_dict.keys()}  ## folds created here

path = "../../../data/raw_data/Features/A/"

images = {}
labels={}
dirs = os.listdir(path)
#print(dirs)
## Create directories for data separation into 'train' and 'validation'    

for val_dir in validation_slides:
    des_path = '../../../data/raw_data/Features/folded_A/'+val_dir+'_val_total_score/'
    try: 
        if os.path.exists(des_path):
            print(os.rmdir(des_path));
        if not os.path.exists(des_path):
            os.makedirs(des_path+'train')
            os.makedirs(des_path+'validation')
    except OSError:
        print('Error: Could not directory of data, it already exists')
    
    try:
        if not os.path.exists(des_path+val_dir+'_val_total_score_cropped/'):
            os.makedirs(des_path+val_dir+'_val_total_score_cropped/train')
            os.makedirs(des_path+val_dir+'_val_total_score_cropped/validation')
    except OSError:
        print('Error: Could not directory of data, it already exists')
    
    # Boundaries for severity categories    
    min_score = 0
    max_score = 8
    
    
    # Create label file with scoring according to image name with score appended to end
    
    for group in dirs:
        
        val_samples_nr = fold_dict[val_dir][group]
        train_label = []
        validation_label = []
    
        train_samples = []
        validation_samples = []
    
        idx = 0
        images  = glob.glob(path+group+'/*')
      # Check image slide_nr and compare to val_samples above, if match add to val
        for img in sorted(images):
            
        
            slide_nr = int(img.split('/')[-1].split('.')[0].split('_')[1])
            img_label = str(img.split('_score_')[1].replace('.tif','').replace('.TIF',''))
            print(img_label)
            if slide_nr in val_samples_nr: #and fold==val_dir:
                print(shutil.copyfile(img,des_path+'validation/' + img.split('/')[-1].split('\\')[1]))
                validation_label.append(img_label)
                validation_samples.append(img.split('/')[-1].split('\\')[1])
            else:
                print(shutil.copyfile(img,des_path+'train/' + img.split('/')[-1].split('\\')[1]))
                train_label.append(img_label)
                train_samples.append(img.split('/')[-1].split('\\')[1])
        
            #idx += 1
    
          ## The list of train images are saved with corresponding labels
        with open (des_path+'train/train_labels.txt','a+') as f:
            for k,n in zip(train_samples,train_label):
                print(k,n,sep=' ',file=f)
        f.close()
          
      ## The list of val images are saved with corresponding labels
        with open (des_path+'validation/validation_labels.txt','a+') as f:    
            for i,b in zip(validation_samples,validation_label):
                print(i,b,sep=' ',file=f)
                
        f.close()

Error: Could not directory of data, it already exists
1.75
../../../data/raw_data/Features/folded_A/1st_val_total_score/train/Slide_9._10_1_191129_-_Pig_2_control_MV_base_line_mv_ecmo_control_score_1.75.tif
1.75
../../../data/raw_data/Features/folded_A/1st_val_total_score/train/Slide_9._1_1_191129_-_Pig_2_control_MV_base_line_mv_ecmo_control_score_1.75.tif
1.75
../../../data/raw_data/Features/folded_A/1st_val_total_score/train/Slide_9._2_1_191129_-_Pig_2_control_MV_base_line_mv_ecmo_control_score_1.75.tif
1.75
../../../data/raw_data/Features/folded_A/1st_val_total_score/train/Slide_9._3_1_191129_-_Pig_2_control_MV_base_line_mv_ecmo_control_score_1.75.tif
1.75
../../../data/raw_data/Features/folded_A/1st_val_total_score/train/Slide_9._4_1_191129_-_Pig_2_control_MV_base_line_mv_ecmo_control_score_1.75.tif
1.75
../../../data/raw_data/Features/folded_A/1st_val_total_score/train/Slide_9._5_1_191129_-_Pig_2_control_MV_base_line_mv_ecmo_control_score_1.75.tif
1.75
../../../data/raw_data/Featu

In [36]:

import cv2
import math

tile_size = (224,224)
offset = (224,224)



validation_slides = ['1st','2nd','3rd']
for val_dir in validation_slides:
    main_path = '../../../data/raw_data/Features/folded_A/'+val_dir+'_val_total_score/'
    dest_path = '../../../data/raw_data/Features/folded_A/'+val_dir+'_val_total_score/'+val_dir+'_val_total_score_cropped/'
    
    dirs = os.listdir(main_path)
    print('Current directories \n')
    print(dirs)
    print(os.getcwd())
    
    # If one wants to include entire image but with white space change floor to ceil below
    # Iterate over each image and cut out as many 224 x 224 tiles as possible
    for d in dirs:
        #if '.' in d: #catch potential residual files from .ipynb
        #  continue
        if d in ['train','validation']:
            label_file = main_path+d+'/'+d+'_labels.txt'
            new_label_file = main_path+d+'/'+d+'_newlabels.txt'
        
            count = 0
            with open(label_file,'r') as f:
                with open(new_label_file,'+a') as f_out:
                    for line in f:
                        print('Cropping image ' + str(count))
                        img_name = line.split('/')[-1].split()[0]
                        img_label = line.split('/')[-1].split()[1]
                        img = cv2.imread(main_path+d+'/'+img_name) # (586, 1335, 3)
                        
                        img_shape = img.shape
                        base_name = img_name.replace(".tif","___")
                        for i in range(int(math.floor(img_shape[0]/(offset[1] * 1.0)))):
                            for j in range(int(math.floor(img_shape[1]/(offset[0] * 1.0)))):
                                cropped_img = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
                                cv2.imwrite(dest_path+ d +"/"+ base_name + str(i) + "_" + str(j) + ".tif", cropped_img)
                                print(base_name + str(i) + "_" + str(j) + ".tif",img_label,file=f_out)
                        count += 1
    print("Tile splitting done!")

Current directories 

['1st_val_total_score_cropped', 'train', 'validation']
C:\ImageProject_files\Histology\code\Three_class_files\notebook\Regression\notebooks
Cropping image 0
Cropping image 1
Cropping image 2
Cropping image 3
Cropping image 4
Cropping image 5
Cropping image 6
Cropping image 7
Cropping image 8
Cropping image 9
Cropping image 10
Cropping image 11
Cropping image 12
Cropping image 13
Cropping image 14
Cropping image 15
Cropping image 16
Cropping image 17
Cropping image 18
Cropping image 19
Cropping image 20
Cropping image 21
Cropping image 22
Cropping image 23
Cropping image 24
Cropping image 25
Cropping image 26
Cropping image 27
Cropping image 28
Cropping image 29
Cropping image 30
Cropping image 31
Cropping image 32
Cropping image 33
Cropping image 34
Cropping image 35
Cropping image 36
Cropping image 37
Cropping image 38
Cropping image 39
Cropping image 40
Cropping image 41
Cropping image 42
Cropping image 43
Cropping image 44
Cropping image 45
Cropping image 46
Cr